### 文本聚类

### **tfidf+kmeans, Single-pass clustering, dbscan, 文本层次聚类**

在商业语境下也叫“典型意见挖掘”  
从大量无标签的文本数据中发现其潜在的结构，根据文本数据的结构特征“归纳”出若干个主题或者意见。  
   
一般情况下文本聚类包含以下步骤：  
step1: 文本预处理（切词、去除停用词、数字归一化等）  
step2: 构建向量空间模型(Vector Space Model，VSS)  
step3: 使用LSA / SVD对文档向量降维（可选，当维度过高时可以选用）  
step4: 使用肘部方法（Elbow Method） 去发现最优的聚类数量  
step5: 按照得出的最优聚类数，使用K-means或Minibatch K-means进行文本聚类

#### TFIDF + KMeans 

In [ ]:
# 文本特征提取 tfidf
vectorizer = NumberNormalizingVectorizer(max_df=0.5, max_features=n_features,
                                         min_df=2, stop_words=stwlist,ngram_range=(1, 2),
                                         use_idf=opts.use_idf)

X = vectorizer.fit_transform(data['正文切词'])

#Vectorizer的结果被归一化，这使得KMeans表现为球形k均值（Spherical K-means）以获得更好的结果。 
#由于LSA / SVD结果并未标准化，我们必须重做标准化。
svd = TruncatedSVD(opts.n_components)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

X = lsa.fit_transform(X)

explained_variance = svd.explained_variance_ratio_.sum()
print("SVD解释方差的step: {}%".format(int(explained_variance * 100)))

使用肘部方法（Elbow Method） 发现最佳聚类数

In [ ]:
n_clusters= 40

wcss = []
for i in range(1,n_clusters):
if opts.minibatch:
        km = MiniBatchKMeans(n_clusters=i, init='k-means++', n_init=2,n_jobs=-1,
                         init_size=1000, batch_size=1500, verbose=opts.verbose)
else:
        km = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=2,n_jobs=-1,
                verbose=opts.verbose)
    km.fit(X)
    wcss.append(km.inertia_)
plt.plot(range(1,n_clusters),wcss)
plt.title('肘 部 方 法')
plt.xlabel('聚类的数量')
plt.ylabel('wcss')
plt.show()

很多情况下，并不会有明显的拐点出现  
笔者比较提倡的做法还是从实际问题出发，人工指定比较合理的K值，通过多次随机初始化聚类中心选取比较满意的结果。  
在这里，笔者选择30作为聚类数，先试试效果，如果效果不好，再试试18、25等聚类数量。

In [ ]:
km = KMeans(n_clusters=true_k, init='k-means++', max_iter=300, n_init=5,n_jobs=-1,
            verbose=opts.verbose)

km.fit(X)

print("Homogeneity值: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
print("Completeness值: %0.3f" % metrics.completeness_score(labels, km.labels_))
print("V-measure值: %0.3f" % metrics.v_measure_score(labels, km.labels_))
print("Adjusted Rand-Index值: %.3f"% metrics.adjusted_rand_score(labels, km.labels_))
print("Silhouette Coefficient值: %0.3f"% metrics.silhouette_score(X, km.labels_, sample_size=1000))

#### Single-pass clustering 单遍聚类

它是一种简洁且高效的文本聚类算法。在文本主题聚类中，Single-pass聚类算法比K-means来的更为有效。
Single-pass聚类算法不需要指定类目数量，可以通过设定相似度阈值来限定聚类数量。

In [ ]:
import numpy as np
import math
import jieba
import json
from gensim import corpora, models, similarities, matutils
from smart_open import  smart_open
import pandas as pd
from pyltp import SentenceSplitter  #按标点切分语句
from  textrank4zh import TextRank4Keyword,TextRank4Sentence #关键词和关键句提取
from tkinter import _flatten   #用于将嵌套列表压成一层

class Single_Pass_Cluster(object):
    def __init__(self, 
                 filename, 
                 stop_words_file= '停用词汇总.txt',
                 theta = 0.5):

        self.filename = filename
        self.stop_words_file = stop_words_file
        self.theta = theta 

    def loadData(self,filename):
        '''以列表的形式读取文档'''
        Data = []
        i = 0
        with smart_open(self.filename,encoding='utf-8') as f:    
            #鉴于有些文档较长，包含多个语义中心，因此按语句结束标点进行切割获取表意单一的句子产生的聚类效果会更好    
            texts = [list(SentenceSplitter.split(i.strip().strip('\ufeff'))) for i in f.readlines()]
            print('未切割前的语句总数有{}条...'.format(len(texts)))
            print ("............................................................................................")  
            texts = [i.strip() for i in list(_flatten(texts)) if len(i)>5]
            print('切割后的语句总数有{}条...'.format(len(texts)))
            for line in texts:
                i  += 1
                Data.append(line )
        return Data

    def word_segment(self,texts):
        '''对语句进行分词，并去掉常见无意义的高频词（停用词）'''
        stopwords = [line.strip() for line in open( self.stop_words_file,encoding='utf-8').readlines()]
        word_segmentation = []
        words = jieba.cut(texts)
        for word in words:
            if word == ' ':
                continue
            if word not in stopwords:
                word_segmentation.append(word)
        return word_segmentation

    def get_Tfidf_vector_representation(self,word_segmentation,pivot= 10, slope = 0.1):
        '''采用VSM(vector space model)得到文档的空间向量表示，也可以doc2vec等算法直接获取句向量'''
        dictionary = corpora.Dictionary(word_segmentation)  #获取分词后词汇和词汇id的映射关系，形成字典
        corpus = [dictionary.doc2bow(text) for text in word_segmentation]   #得到语句的向量表示
        tfidf = models.TfidfModel(corpus,pivot=pivot, slope =slope)      #进一步获取语句的TF-IDF向量表示
        corpus_tfidf = tfidf[corpus]
        return corpus_tfidf

    def getMaxSimilarity(self,dictTopic, vector):
        '''计算新进入文档和已有文档的文本相似度，这里采用的是cosine余弦相似度，还可以试试kullback_leibler, jaccard, hellinger等'''
        maxValue = 0
        maxIndex = -1
        for k,cluster in dictTopic.items():
            oneSimilarity = np.mean([matutils.cossim(vector, v) for v in cluster])
            if oneSimilarity > maxValue:
                maxValue = oneSimilarity
                maxIndex = k
        return maxIndex, maxValue

    def single_pass(self,corpus,texts,theta):
        dictTopic = {}
        clusterTopic = {}
        numTopic = 0 
        cnt = 0
        for vector,text in zip(corpus,texts): 
            if numTopic == 0:
                dictTopic[numTopic] = []
                dictTopic[numTopic].append(vector)
                clusterTopic[numTopic] = []
                clusterTopic[numTopic].append(text)
                numTopic += 1
            else:
                maxIndex, maxValue = self.getMaxSimilarity(dictTopic, vector)
                # 以第一篇文档为种子，建立一个主题，将给定语句分配到现有的、最相似的主题中
                if maxValue > theta:
                    dictTopic[maxIndex].append(vector)
                    clusterTopic[maxIndex].append(text)

                # 或者创建一个新的主题
                else:
                    dictTopic[numTopic] = []
                    dictTopic[numTopic].append(vector)
                    clusterTopic[numTopic] = []
                    clusterTopic[numTopic].append(text)
                    numTopic += 1
            cnt += 1
            if cnt % 1000 == 0:
                print ("processing {}...".format(cnt))
        return dictTopic, clusterTopic  

    def fit_transform(self,theta=0.5):
        '''综合上述的函数，得出最终的聚类结果：包括聚类的标号、每个聚类的数量、关键主题词和关键语句'''
        datMat = self.loadData(self.filename)  
        word_segmentation = []
        for i in range(len(datMat)):
            word_segmentation.append(self.word_segment(datMat[i]))          
        print ("............................................................................................")  
        print('文本已经分词完毕 !')

        #得到文本数据的空间向量表示
        corpus_tfidf = self.get_Tfidf_vector_representation(word_segmentation)
        dictTopic, clusterTopic = self.single_pass(corpus_tfidf, datMat, theta)
        print ("............................................................................................")  
        print( "得到的主题数量有: {} 个 ...".format(len(dictTopic)))
        print ("............................................................................................\n")  
        #按聚类语句数量对聚类结果进行降序排列，找到重要的聚类群
        clusterTopic_list = sorted(clusterTopic.items(),key=lambda x: len(x[1]),reverse=True)
        for k in clusterTopic_list:
            cluster_title = '\n'.join(k[1]) 
            # 得到每个聚类中的主题关键词
            word = TextRank4Keyword()
            word.analyze(''.join(self.word_segment(''.join(cluster_title))),window = 5,lower = True)
            w_list = word.get_keywords(num = 10,word_min_len = 2)
           # 得到每个聚类中的关键主题句TOP3
            sentence = TextRank4Sentence()
            sentence.analyze(' '.join(k[1]) ,lower = True)
            s_list = sentence.get_key_sentences(num = 3,sentence_min_len = 3)
            print ("【主题索引】:{} \n【主题语量】：{} \n【主题关键词】：{} \n【主题中心句】 ：\n{}".format(k[0],len(k[1]),','.join([i.word for i in w_list]),'\n'.join([i.sentence for i in s_list])))

#### dbscan聚类

由于 DBSCAN不能很好反映高维数据，所以对抽取的特征进行降维是很有必要的.这里采用的是LSA降维，暂时设定15维

In [ ]:
vectorizer = TfidfVectorizer(max_df=0.5, 
                             max_features=40000,
                             min_df=5, 
                             stop_words=stwlist,ngram_range=(1, 2),
                             use_idf=True)

X = vectorizer.fit_transform(data['正文切词'])

svd = TruncatedSVD(15)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

X = lsa.fit_transform(X)

db = DBSCAN(eps=0.2, min_samples=4).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True

聚类数及噪点计算：

In [ ]:
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('聚类数：',n_clusters_)
print('噪点数：',n_noise_)

#### 文本层次聚类

In [ ]:
vec = TfidfVectorizer()
X = vec.fit_transform(sentences)

# 计算成对的余弦相似度，也就是计算出一个相似度矩阵
sims = cosine_similarity(X)
# 对相似度矩阵中每个cell的值保留5位小数，确保精度不损失太多：
similarity = np.round(sims, decimals = 5)

# 
cluster = AgglomerativeClustering(
                                  n_clusters = None,
                                  distance_threshold= 0,
                                  affinity = "cosine",
                                  linkage = "average"
                                                    )  
cluster.fit(similarity)

绘制树状图的函数

In [ ]:
def plot_dendrogram(model, **kwargs):

      # 建立邻接矩阵(linkage matrix)，然后绘制树状图（dendrogram）

   # 创建每个节点下的样本数
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # 绘制对应的树状图
    dendrogram(linkage_matrix, **kwargs)

In [ ]:
#绘制树状图的前14层
plot_dendrogram(cluster, 
                truncate_mode='level',
                p=14, 
                leaf_font_size=15,
               )

plt.xlabel("节点中的点的数量（如果没有括号，则为语句对应的索引）",size = 25)
plt.show()